<a href="https://colab.research.google.com/github/rafa-cc/Proyecto-Final-R/blob/main/Proyecto_Final_R_Emp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [225]:
import pandas as pd
import numpy as np
import statistics
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import plotly.graph_objects as go

In [234]:
# Cargar los datos guardados en el análisis hecho con R de la sección _Análisis de empaquetado_
df = pd.read_csv('https://raw.githubusercontent.com/rafa-cc/Proyecto-Final-R/main/data.csv')
df.head()

,Length,Width,Height
0,0.1,5.1,6.2
1,0.2,4.4,6.8
2,0.3,0.3,0.7
3,0.3,0.3,0.7
4,0.3,2.9,4.3


Los datos se normalizan respecto a la mediana pues al final lo que caracteriza a cada forma es la distancia de los valores extremos respecto a ese punto medio, por ello fue importane ordenar por filas de menor a mayor cuando se manipuló con `R`

In [227]:
def normal(triada):
    # Calcular la mediana de la triada
    med = statistics.median(triada)

    # Normalizar cada número en la triada respecto a la mediana
    triada_normalizada = [round(x / med,2) for x in triada]

    return triada_normalizada

In [228]:
# Crear una columna con una lista de las dimensiones normalizadas
df['normal'] = df.apply(normal, axis=1)
df.head()

,Length,Width,Height,normal
0,0.1,5.1,6.2,"[0.02, 1.0, 1.22]"
1,0.2,4.4,6.8,"[0.05, 1.0, 1.55]"
2,0.3,0.3,0.7,"[1.0, 1.0, 2.33]"
3,0.3,0.3,0.7,"[1.0, 1.0, 2.33]"
4,0.3,2.9,4.3,"[0.1, 1.0, 1.48]"


El modelo se entrena con los valores posibles que pueden tener las formas de los objetos. _Por ejemplo un cubo puede tener su mínimo entre $\sf[0.66,1)$ y su máximo entre $\sf(1,1.33]$_. Entonces se colocan algunas combinaciones para que el modelo considere ese rango de opciones.

In [229]:
# Ejemplo de datos etiquetados para entrenar el modelo
data = [
    ((0.66, 1.0, 1.0), 'Cubo'),
    ((1.0, 1.0, 1.0), 'Cubo'),
    ((1.0, 1.0, 1.33), 'Cubo'),
    ((0.66, 1.0, 1.33), 'Cubo'),
    ((0.88, 1.0, 1.0), 'Cubo'),
    ((1.0, 1.0, 1.0), 'Cubo'),
    ((1.0, 1.0, 1.11), 'Cubo'),
    ((0.88, 1.0, 1.11), 'Cubo'),
    ((0.88, 1.0, 1.33), 'Cubo'),
    ((0.66, 1.0, 1.11), 'Cubo'),

    ((0.11, 1.0, 1.0), 'Pizza'),
    ((0.33, 1.0, 1.0), 'Pizza'),
    ((0.4, 1.0, 1.0), 'Pizza'),
    ((0.11, 1.0, 1.33), 'Pizza'),
    ((0.33, 1.0, 1.33), 'Pizza'),
    ((0.4, 1.0, 1.33), 'Pizza'),
    ((0.4, 1.0, 1.11), 'Pizza'),
    ((0.33, 1.0, 1.11), 'Pizza'),
    ((0.11, 1.0, 1.11), 'Pizza'),

    ((1.0, 1.0, 2.0), 'Leche'),
    ((1.0, 1.0, 1.66), 'Leche'),
    ((1.0, 1.0, 1.4), 'Leche'),
    ((0.88, 1.0, 2.0), 'Leche'),
    ((0.88, 1.0, 1.66), 'Leche'),
    ((0.88, 1.0, 1.4), 'Leche'),
    ((0.66, 1.0, 2.0), 'Leche'),
    ((0.66, 1.0, 1.66), 'Leche'),
    ((0.66, 1.0, 1.4), 'Leche'),

    ((1.0, 1.0, 2.11), 'Estaca'),
    ((1.0, 1.0, 2.66), 'Estaca'),
    ((1.0, 1.0, 2.4), 'Estaca'),
    ((0.88, 1.0, 2.0), 'Estaca'),
    ((0.88, 1.0, 2.66), 'Estaca'),
    ((0.88, 1.0, 2.4), 'Estaca'),
    ((0.66, 1.0, 2.0), 'Estaca'),
    ((0.66, 1.0, 2.66), 'Estaca'),
    ((0.66, 1.0, 2.4), 'Estaca'),

    ((0.11, 1.0, 1.4), 'Tabla'),
    ((0.11, 1.0, 1.6), 'Tabla'),
    ((0.11, 1.0, 1.8), 'Tabla'),
    ((0.11, 1.0, 2.0), 'Tabla'),
    ((0.11, 1.0, 2.4), 'Tabla'),
    ((0.11, 1.0, 2.6), 'Tabla'),
    ((0.33, 1.0, 1.4), 'Tabla'),
    ((0.33, 1.0, 1.6), 'Tabla'),
    ((0.33, 1.0, 1.8), 'Tabla'),
    ((0.33, 1.0, 2.0), 'Tabla'),
    ((0.33, 1.0, 2.4), 'Tabla'),
    ((0.33, 1.0, 2.6), 'Tabla'),
    ((0.4, 1.0, 1.4), 'Tabla'),
    ((0.4, 1.0, 1.6), 'Tabla'),
    ((0.4, 1.0, 1.8), 'Tabla'),
    ((0.4, 1.0, 2.0), 'Tabla'),
    ((0.4, 1.0, 2.4), 'Tabla'),
    ((0.4, 1.0, 2.6), 'Tabla'),

    ((0.6, 1.0, 1.0), 'Desco'),
    ((0.6, 1.0, 1.11), 'Desco'),
    ((0.6, 1.0, 1.33), 'Desco'),
    ((0.6, 1.0, 1.4), 'Desco'),
    ((0.6, 1.0, 1.6), 'Desco'),
    ((0.6, 1.0, 2.0), 'Desco'),
    ((0.5, 1.0, 1.0), 'Desco'),
    ((0.5, 1.0, 1.11), 'Desco'),
    ((0.5, 1.0, 1.33), 'Desco'),
    ((0.5, 1.0, 1.4), 'Desco'),
    ((0.5, 1.0, 1.4), 'Desco'),
    ((0.5, 1.0, 1.6), 'Desco'),
    ((0.5, 1.0, 2.0), 'Desco'),
]

# Separar triadas y etiquetas
X = [entry[0] for entry in data]
y = [entry[1] for entry in data]

# Dividir el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo
model = SVC()
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy}')

Precisión del modelo: 0.7142857142857143


In [230]:
# Crear una función para aplicar el modelo a cada triada
def clasificar_triada(triada):
    etiqueta = model.predict([triada])[0]
    return etiqueta

# Aplicar la función a la columna 'normal' y crear una nueva columna 'etiqueta'
df['etiqueta'] = df['normal'].apply(clasificar_triada)

Se calcula la distancia al origen pues, si se considera el origen como una esquina de la caja, el punto será la esquina opuesta y esto es directamente proporcional al volumen de la caja (_entre más se aleja el punto de la caja, más grande es ésta_)

In [231]:
# Crear la columna 'distance' las cuales son la distancia al origen de cada punto
df['distance'] = np.sqrt(df['Length']**2 + df['Width']**2 + df['Height']**2)

print(round(df['distance'].quantile(0.95),1))
# Filtrar los puntos con ditancia menor o igual a 21 pues ahi se encuentra el 95% de los datos
df_temp = df.query('distance <= 21')

20.5


In [232]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=df['distance']))

# Líneas verticales para la media
fig.add_shape(
    go.layout.Shape(
        type="line",
        x0=df['distance'].mean(),
        x1=df['distance'].mean(),
        y0=0,
        y1=180,
        line=dict(color="red", width=2),
    )
)

# Línea punteada que representa el cuantil 95
fig.add_shape(
    go.layout.Shape(
        type="line",
        x0=df['distance'].quantile(0.95),
        x1=df['distance'].quantile(0.95),
        y0=0,
        y1=180,
        line=dict(color="blue", width=2, dash="dash"),
    )
)

fig.update_layout(
    title='Histograma de Distribución de la Distancia al Origen',
    xaxis=dict(title='Cantidad de datos'),
    yaxis=dict(title='Frecuencia'),
)

fig.show()

Ahora se puede ver cómo los datos fueron categorizados y cada etiqueta forma una especie de línea.

Notar como el conjunto de datos parece una pirámide.

In [233]:
import plotly.graph_objects as go

fig = go.Figure()

for cluster in df_temp['etiqueta'].unique():
    df_cluster = df_temp[df_temp['etiqueta'] == cluster]
    fig.add_trace(go.Scatter3d(
        x=df_cluster['Length'],
        y=df_cluster['Width'],
        z=df_cluster['Height'],
        mode='markers',
        marker=dict(
            size=1,
            opacity=0.9
        ),
        name=f'Cluster {cluster}'
    ))

fig.update_layout(scene=dict(
    xaxis_title='Length',
    yaxis_title='Width',
    zaxis_title='Height'
))

fig.show()